**计算Shannon-Cosine wavelet 尺度函数的导数（一阶和二阶）**

In [6]:
import sys, os
sys.path.append(os.path.dirname(os.path.realpath('../utils')))
import numpy as np
import sys
from utils.aiwt import cjkfun, z_g, CompressInterp, adapt_CjJ4

In [7]:
j0 = 0
J = 15
a = 0
b = 2
deltax = (b-a)/(2**J)
XJ0 = np.arange(a, b+deltax, deltax).reshape(1,-1)
V0 = np.sin(np.pi * XJ0)
eps0 = 0.005

M = 1
CC = 1

cjk, cjknum, zJc, zJcnum = cjkfun(V0, J, j0, a, b, eps0)
gjk, gjknum, gJc, gJcnum = z_g(j0, J, cjk, cjknum, zJc, zJcnum, a, b, M, CC)
XJ = a + deltax * gJc
V0 = CompressInterp(XJ0,V0,XJ)
C = adapt_CjJ4(J, j0, gjk, gjknum, gJc, gJcnum, a, b)

In [8]:
delta = (b - a) / (2 ** J)
xk = a + gJc * delta
xk = xk.reshape((1,-1))
k = np.arange(2 ** j0+1).reshape((1,-1))

In [9]:
def sad1x_Gauss(deltaxk, delta, R):
    mm = (R**2) * (delta**2)
    t = np.pi * deltaxk / delta
    s = np.sin(t)
    c = np.cos(t)
    e = np.exp(-np.square(deltaxk) / (2 * mm))
    se = s * e
    ce = c * e
    d1theta = ce/deltaxk - se*delta/(np.pi * deltaxk**2) - se * delta / (np.pi * mm)
    return d1theta


def sad2x_Gauss(deltaxk, delta, R):
    mm = (R**2) * (delta**2)
    t = np.pi * deltaxk / delta
    s = np.sin(t)
    c = np.cos(t)
    e = np.exp(-np.square(deltaxk) / (2 * mm))
    se = s * e
    ce = c * e
    d2theta = -np.pi * se / (delta * deltaxk) - 2 * ce / (deltaxk ** 2) - 2 * ce / mm + \
              2 * se * delta / (np.pi * (deltaxk ** 3)) + se * delta / (np.pi * mm * deltaxk) + \
              deltaxk * se * delta / (np.pi * mm * mm)
    return d2theta

In [10]:
def dxfai0_Shannon_Cosine(j,k,xmin,xmax,x):
    R = 3.2
    delta = (xmax - xmin) / (2 ** j)
    xk = xmin + delta * k
    t = np.ones((1,k.shape[1]))
    if x.shape[0] == 1:
        x = x.T
    x = x * t
    t = np.ones((x.shape[0], 1))
    if xk.shape[1] == 1:
        xk = xk.T
    xk = t * xk
    deltaxk = x - xk
    a = (abs(deltaxk) < sys.float_info.epsilon)
    mm = R * delta
    d1theta = 0 * a
    d2theta = -(3 + (np.pi**2)*(R**2)) / (3 * mm ** 2) * a
    v_sad1x_Gauss = np.vectorize(sad1x_Gauss)
    v_sad2x_Gauss = np.vectorize(sad2x_Gauss)
    idx = np.where(deltaxk != 0)
    c1 = np.zeros_like(deltaxk)
    c1[idx] = v_sad1x_Gauss(deltaxk[idx], delta, R)
    c2 = np.zeros_like(deltaxk)
    c2[idx] = v_sad2x_Gauss(deltaxk[idx], delta, R)
    d1theta = d1theta + c1
    d2theta = d2theta + c2
    return d1theta, d2theta

In [11]:
ss1, ss2 = dxfai0_Shannon_Cosine(j0,k,a,b,xk)